## Context

Here, I will implement several neural network models to classify images from the [Animals-10](https://www.kaggle.com/datasets/alessiocorrado99/animals10) database.

The Animals-10 database contains approximately 28,000 medium-quality animal images, divided into 10 categories, each stored in a separate folder. The number of images per category varies between 2,000 and 5,000.

Install and load relevant libraries

In [1]:
# Instalamos la última versión de Tensorflow (con CUDA)
%pip install tensorflow[and-cuda]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 85.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 70.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 72.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 MB 17.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 6.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [2]:
# Import Tensorflow
import tensorflow as tf
print("TF version   : ", tf.__version__)

# Available GPU
print("GPU available: ", tf.config.list_physical_devices('GPU'))

# Keras version
import keras
print("Keras version   : ", keras.__version__)

TF version   :  2.16.1
GPU available:  []
Keras version   :  3.3.3


In [3]:
# Import Keras relevant elements
from keras.utils import image_dataset_from_directory
from keras.layers import (
    GlobalAveragePooling2D, Flatten, Input, 
    Dense, Dropout, Conv2D, Conv2DTranspose, BatchNormalization, 
    MaxPooling2D, UpSampling2D, Rescaling, Resizing,
    RandomFlip, RandomRotation, RandomZoom, RandomContrast)
from keras.callbacks import (EarlyStopping, ReduceLROnPlateau)
from keras.optimizers import (Adam, RMSprop)
from keras import Sequential, Model

In [4]:
# Import other relevant libraries
import cv2
from PIL import Image
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import seaborn as sns

## 1. Data loading and pre-processing

Images can be found in this [link](https://www.kaggle.com/datasets/svalbuenaa/images-deep-learning-cnns)

In [5]:
# Extract names of the clases
# For the train dataset
train_dir_path = "/kaggle/input/images-deep-learning-cnns/images/train"
train_classes = os.listdir(train_dir_path)
print(train_classes)

['spider', 'horse', 'butterfly', 'dog', 'chicken', 'elephant', 'sheep', 'cow', 'squirrel', 'cat']


In [6]:
# For the test dataset
test_dir_path = "/kaggle/input/images-deep-learning-cnns/images/test"
test_classes = os.listdir(test_dir_path)
print(test_classes)

['spider', 'horse', 'butterfly', 'dog', 'chicken', 'elephant', 'sheep', 'cow', 'squirrel', 'cat']
